In [1]:
import sklearn

In [2]:
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier

import pandas as pd
import numpy as np

In [7]:
X, y = make_classification()
pdf = pd.DataFrame(X)
pdf.columns = ['c{}'.format(x) for x in range(X.shape[1])]

In [8]:
X.shape

(100, 20)

In [9]:
X1 = pdf[['c{}'.format(x) for x in range(10, 20)]]
X2 = pdf[['c{}'.format(x) for x in range(10)]]

In [37]:
"""
Implement DPP version that is similar to what is done above
"""

class DPPRegressor(SGDRegressor):
    def __init__(self, loss="squared_loss", penalty="l2", alpha=0.0001,
                 l1_ratio=0.15, fit_intercept=True, max_iter=None, tol=None,
                 shuffle=True, verbose=0, epsilon=0.1,
                 random_state=None, learning_rate="invscaling", eta0=0.01,
                 power_t=0.25, warm_start=False, average=False, n_iter=None, reg_penalty=None):
        super(GraftingRegressor, self).__init__(loss=loss, penalty=penalty,
                                           alpha=alpha, l1_ratio=l1_ratio,
                                           fit_intercept=fit_intercept,
                                           max_iter=max_iter, tol=tol,
                                           shuffle=shuffle,
                                           verbose=verbose,
                                           epsilon=epsilon,
                                           random_state=random_state,
                                           learning_rate=learning_rate,
                                           eta0=eta0, power_t=power_t,
                                           warm_start=warm_start,
                                           average=average, n_iter=n_iter)
        self.filter_cols = []
        self.base_shape = None
        self.reg_penalty = reg_penalty if reg_penalty is not None else l1_ratio
    
    def _fit_columns(self, X, return_x=True):
        """
        Method filter through "unselected" columns. The goal of this 
        method is to filter any uninformative columns.
        
        This will be selected based on index only?
        
        If return_x is false, it will only return the boolean mask.
        """
        import pandas
        bool_mask = np.ones((X.shape[1],), dtype=np.bool)
        if len(self.filter_cols) == 0:
            if return_x:
                return X
            else:
                return bool_mask
        # otherwise...
        bool_mask[self.filter_cols] = False
        if not return_x:
            return bool_mask
        if type(X) is pandas.core.frame.DataFrame:
            return X[X.columns[bool_mask]]
        else:
            return X[:, bool_mask]
    
    def _reg_penalty(self):
        bool_mask = np.zeros((self.coef_.shape[0],), dtype=np.bool)
        keep_cols = np.argwhere(np.abs(self.coef_) > self.reg_penalty)        
        mask = np.array(list(set(keep_cols.flatten().tolist() + list(range(self.base_shape)))))
        self.coef_ = self.coef_[mask]
        bool_mask[mask] = True
        self.filter_cols = np.argwhere(~bool_mask).flatten().tolist()        
    
    def _partial_grafting_fit(self, X, y):
        """
        Partial fit grafting method to expand the coefficient listing
        to taking into account new coefficients
        """
        # require to know the base shape to determine/
        # check for irrelevant columns in the future.
        self.base_shape = self.coef_.shape[0]
        
        X = self._fit_columns(X)
        n_samples, n_features = X.shape
        coef_list = np.zeros(n_features, dtype=np.float64, order="C")
        coef_list[:self.coef_.shape[0]] = self.coef_.copy()
        self.coef_ = coef_list.copy()
        
    def partial_fit(self, X, y, sample_weight=None):
        self._partial_grafting_fit(X, y)
        super(GraftingRegressor, self).partial_fit(X, y, sample_weight=None)  
        
        # update parameters based on weight of regularizer penalty
        self._reg_penalty()
        return self
    
    def predict(self, X):
        X = self._fit_columns(X)
        return super(GraftingRegressor, self).predict(X)        

In [38]:
model = GraftingClassifier(max_iter=1000)
model.fit(X1, y)

GraftingClassifier(alpha=0.0001, average=False, class_weight=None,
          epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
          learning_rate='optimal', loss='log', max_iter=1000, n_iter=None,
          n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
          reg_penalty=0.15, shuffle=True, tol=None, verbose=0,
          warm_start=False)

In [39]:
model.coef_.shape

(1, 10)

In [40]:
model.partial_fit(pdf, y)

GraftingClassifier(alpha=0.0001, average=False, class_weight=None,
          epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
          learning_rate='optimal', loss='log', max_iter=1000, n_iter=None,
          n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
          reg_penalty=0.15, shuffle=True, tol=None, verbose=0,
          warm_start=False)

In [41]:
model.coef_.shape

(1, 20)

In [42]:
model.predict_proba(pdf)

array([[  5.37347944e-14,   1.00000000e+00],
       [  1.00000000e+00,   4.51335214e-21],
       [  5.39579439e-05,   9.99946042e-01],
       [  9.99976621e-01,   2.33790698e-05],
       [  1.55368414e-03,   9.98446316e-01],
       [  9.75326194e-01,   2.46738060e-02],
       [  8.49456302e-08,   9.99999915e-01],
       [  9.99978273e-01,   2.17274956e-05],
       [  9.70334924e-14,   1.00000000e+00],
       [  9.66951208e-01,   3.30487919e-02],
       [  1.00000000e+00,   3.78305735e-13],
       [  9.71810374e-01,   2.81896263e-02],
       [  9.29884757e-01,   7.01152432e-02],
       [  6.86451926e-01,   3.13548074e-01],
       [  1.81199463e-04,   9.99818801e-01],
       [  3.74383014e-03,   9.96256170e-01],
       [  3.00435295e-02,   9.69956471e-01],
       [  4.14297553e-01,   5.85702447e-01],
       [  5.72559674e-02,   9.42744033e-01],
       [  9.99999240e-01,   7.60215751e-07],
       [  5.45302581e-09,   9.99999995e-01],
       [  9.99999999e-01,   8.65502584e-10],
       [  